In [1]:
import numpy as np
import pandas as pd
from spotlight.evaluation import rmse_score
from spotlight.factorization.explicit import ExplicitFactorizationModel
from spotlight.interactions import Interactions

In [2]:
train_df = pd.read_parquet("../data/interim/20210602_initial_data_children_biography_train_dev_test/children-train.parquet")

In [3]:
user_id_long_to_id = {_id: i for i, _id in enumerate(train_df["Book_id"].unique())}

In [6]:
users = np.asarray([user_id_long_to_id[_id] for _id in train_df["Book_id"]], dtype="int32")
books = train_df["Book_id"].values.astype("int32")
ratings = train_df["Rating"].values.astype("float32")

In [7]:
train = Interactions(users, books, ratings=ratings)

In [ ]:
model = ExplicitFactorizationModel(n_iter=1)
model.fit(train)

In [ ]:
dev_df = pd.read_parquet("../data/interim/20210602_initial_data_children_biography_train_dev_test/children-dev.parquet")

In [ ]:
dev_users = np.asarray([user_id_long_to_id[_id] for _id in dev_df["Book_id"] if _id in dev_df["Book_id"]], dtype="int32")
dev_books = dev_df["Book_id"].values.astype("int32")
dev_ratings = dev_df["Rating"].values.astype("float32")
dev = Interactions(dev_users, dev_books, ratings=dev_ratings)

In [ ]:
rmse = rmse_score(model, dev)